In [3]:
! pip install chembl_webresource_client

  Using cached chembl_webresource_client-0.10.7-py3-none-any.whl (55 kB)
  Using cached requests_cache-0.7.5-py3-none-any.whl (39 kB)
  Using cached easydict-1.9.tar.gz (6.4 kB)
  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached charset_normalizer-2.0.7-py3-none-any.whl (38 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)
  Using cached itsdangerous-2.0.1-py3-none-any.whl (18 kB)
  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6360 sha256=db1905ffbaace9b451e40b22190aef637a304311cb6078cd112a8e6b34c4c474
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\5d\79\e4\4e55effe206295359b37e0f9db3e68a1197ba396682807dadb
Successfully built easydict


In [1]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [2]:
# target identification
target = new_client.target
target_query = target.search('telomerase')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'O14746', 'xref_name': None, 'xre...",Homo sapiens,Telomerase reverse transcriptase,21.0,False,CHEMBL2916,"[{'accession': 'O14746', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Rattus norvegicus,Telomerase reverse transcriptase,21.0,False,CHEMBL3108654,"[{'accession': 'Q673L6', 'component_descriptio...",SINGLE PROTEIN,10116
2,[],Homo sapiens,Prostaglandin E synthase 3,12.0,False,CHEMBL3341580,"[{'accession': 'Q15185', 'component_descriptio...",SINGLE PROTEIN,9606


In [3]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL2916'

In [4]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [5]:
res

[{'activity_comment': None, 'activity_id': 106658, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL813598', 'assay_description': 'Inhibitory activity against telomerase extracted from HCT 116 cell lines', 'assay_type': 'B', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000219', 'bao_label': 'cell-based format', 'canonical_smiles': 'CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NCCCCCCCCCCCCOP(=O)(O)Oc1ccccc1Cl)c1ccccc1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1133972', 'document_journal': 'Bioorg. Med. Chem. Lett.', 'document_year': 2001, 'ligand_efficiency': {'bei': '7.53', 'le': '0.15', 'lle': '-3.24', 'sei': '3.96'}, 'molecule_chembl_id': 'CHEMBL314057', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL1183707', 'pchembl_value': '5.14', 'potential_duplicate': False, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 1568

In [6]:
df = pd.DataFrame.from_dict(res)

In [13]:
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,106658,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,7.3
1,None,106659,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
2,None,107796,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
3,None,107797,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,9.5
4,None,107798,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,None,19072837,[],CHEMBL4362044,Inhibition of telomerase (unknown origin),B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,1.02
891,None,20639331,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,5.2
892,None,20639332,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,6.0
893,None,20639333,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,9.1


In [8]:
df.standard_type.unique()

array(['IC50'], dtype=object)

In [9]:
df.to_csv('C:/Users/DELL/3D Objects/Drug Discovery/bioactivity_data.csv', index=False)

In [12]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

c:\users\dell\anaconda3\envs\drug_dis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,106658,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,7.3
1,None,106659,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
2,None,107796,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
3,None,107797,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,9.5
4,None,107798,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,None,19072837,[],CHEMBL4362044,Inhibition of telomerase (unknown origin),B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,1.02
891,None,20639331,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,5.2
892,None,20639332,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,6.0
893,None,20639333,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,9.1


In [14]:
len(df2.canonical_smiles.unique())

663

In [15]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,106658,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,7.3
1,None,106659,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
2,None,107796,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
3,None,107797,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,9.5
4,None,107798,[],CHEMBL813598,Inhibitory activity against telomerase extract...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,None,19072837,[],CHEMBL4362044,Inhibition of telomerase (unknown origin),B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,1.02
891,None,20639331,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,5.2
892,None,20639332,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,6.0
893,None,20639333,[],CHEMBL4612658,Inhibition of telomerase in human PC3 cell-fre...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Telomerase reverse transcriptase,9606,None,None,IC50,uM,UO_0000065,None,9.1


# Data pre-processing of the bioactivity data
Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame

In [16]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL314057,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,7300.0
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,100000.0
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,100000.0
3,CHEMBL86984,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...,9500.0
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,100000.0
...,...,...,...
890,CHEMBL4453659,CCCc1n[nH]c(C(N)=O)c1NC(=O)/C=C/c1ccco1,1020.0
891,CHEMBL4638841,Cc1cn([C@H]2C[C@H](n3cc(COc4ccc(S(N)(=O)=O)cc4...,5200.0
892,CHEMBL4632724,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...,6000.0
893,CHEMBL4637598,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...,9100.0


In [17]:
df3.to_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)


In [18]:
df4 = pd.read_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv')


In [21]:
bioactivity_threshold = []
for i in df4.standard_value:
    if float(i) >= 10000:
        bioactivity_threshold.append("inactive")
    elif float(i) <= 1000:
        bioactivity_threshold.append("active")
    else:
        bioactivity_threshold.append("intermediate")

In [22]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL314057,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,7300.0,intermediate
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,100000.0,inactive
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,100000.0,inactive
3,CHEMBL86984,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...,9500.0,intermediate
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,100000.0,inactive
...,...,...,...,...
658,CHEMBL4453659,CCCc1n[nH]c(C(N)=O)c1NC(=O)/C=C/c1ccco1,1020.0,intermediate
659,CHEMBL4638841,Cc1cn([C@H]2C[C@H](n3cc(COc4ccc(S(N)(=O)=O)cc4...,5200.0,intermediate
660,CHEMBL4632724,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...,6000.0,intermediate
661,CHEMBL4637598,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...,9100.0,intermediate


In [23]:
df5.to_csv('acetylcholinesterase_03_bioactivity_data_curated.csv', index=False)